# Mini-Project I
During this project, we will practice handling of complex lists and dictionaries in Python. Plus, we will learn how to work with API documentation. Don't be afraid to search for the information in the [**documentation**](https://api.tfl.gov.uk/swagger/ui/index.html?url=/swagger/docs/v1#!/AccidentStats/AccidentStats_Get).

Let's go to the tasks, we have some parsing to do :)!!

In [1]:
# import packages we need (remember what packages we used yesterday during the API session)
# IMPORT HERE
import requests as re
from IPython.display import JSON
import os
import json

## Demo

In [2]:
# don't forget package os
app_id = os.environ['LondonTransitPrimary']
app_key = os.environ['LondonTransitSecondary']
url_append = f'?app_id={app_id}&app_key={app_key}'

In [3]:
# URL
url_root = "https://api.tfl.gov.uk"
url_air = "/AirQuality"

In [4]:
# We send the request to the API
# NOTE: if you don't have your APP_KEY, run this without the url_append
res = re.get(url_root+url_air+url_append)

In [5]:
# We can check if the request was successful
res.status_code

200

In [6]:
# We can export the information that was returned using method .json()
weather = res.json()
JSON(weather)

<IPython.core.display.JSON object>

## Task
Parse the dictionary and print the AirQuality predictions for tomorrow

We have already recieved the json dictionary for air quality. It is just a matter of accessing the information. To do so, we navigate the dictionary layers using sequential keys.

In [7]:
print(weather['currentForecast'][1]['forecastText'])

Widespread frost and freezing fog patches on Saturday and Sunday, slow to clear in places. Dry and sunny otherwise.  Winds are set to remain light and northerly so little in the way of pollution import.&lt;br/&gt;&lt;br/&gt;Air pollution is expected to remain low for the following pollutants:&lt;br/&gt;&lt;br/&gt;Nitrogen Dioxide&lt;br/&gt;Ozone&lt;br/&gt;PM10 Particulate&lt;br/&gt;PM2.5 Particulate&lt;br/&gt;Sulphur Dioxide&lt;br/&gt;&lt;br/&gt;&lt;br/&gt;


## Task
What are the different modes of transport which are operated by Transport for London? How many modes of transport do they support?

Print the list with different modes of transport, plus their count. Example output:
```
[bus, cable-car,.....]
Number of different modes of transport is: xyz
```

We need to search the documentation for correct request.

In [83]:
# These first cells are to make the request and verify the data has been recieved. Later cells will analyze the data. 
url_modes = "/Line/Meta/Modes"

In [88]:
print(url_root+url_modes+"?app_id=[app_id]&app_key=[app_key]")

https://api.tfl.gov.uk/Line/Meta/Modes?app_id=[app_id]&app_key=[app_key]


In [84]:
res_modes = re.get(url_root+url_modes+url_append)
res_modes.status_code

200

In [85]:
modes = res_modes.json()
JSON(modes)

<IPython.core.display.JSON object>

In [86]:
transit = [modes[mode]['modeName'] for mode in range(len(modes))]
print(transit)
print(f'The number of forms of transit supported by the London Transit Authority is {len(transit)}.')

['bus', 'cable-car', 'coach', 'cycle', 'cycle-hire', 'dlr', 'interchange-keep-sitting', 'interchange-secure', 'national-rail', 'overground', 'replacement-bus', 'river-bus', 'river-tour', 'taxi', 'tflrail', 'tram', 'tube', 'walking']
The number of forms of transit supported by the London Transit Authority is 18.


## Task
How many BikePoints in London are operated by Transfor for London? How many docks are in **all** BikePoints? There is the information for empty and full docks for each BikePoint.

In [12]:
url_bike = "/BikePoint"
res_bike = re.get(url_root+url_bike+url_append)
bike_points = res_bike.json()
res_bike.status_code

200

In [89]:
JSON(bike_points)

<IPython.core.display.JSON object>

In [20]:
#  Total number of Bike Points is provided by the number of first entries in the firs layer of the json request.

print(f"The total number of BikePoints is : {len(c)}")

The total number of BikePoints is : 788


In [13]:
#Total number of BikeDocks
docks = [bike_points[point]['additionalProperties'][8]['value'] for point in range(len(bike_points))]
total = 0
for dock in docks:
    total = int(dock) + total
print(f"The total number of BikePoint docks is {total}.")

The total number of BikePoint docks is 20914.


In [15]:
#How many Bike docks are currently broken?
full_docks = [bike_points[point]['additionalProperties'][6]['value'] for point in range(len(docks))]
empty_docks = [bike_points[point]['additionalProperties'][7]['value'] for point in range(len(docks))]
available = 0
in_use = 0
for bike in full_docks:
    available = int(bike) + available
for empty in empty_docks:
    in_use = int(empty) + in_use
broken = total - available - in_use
print(f"The number of currently broken docks is {broken}")

The number of currently broken docks is 459


## Task
How many tube and bus lines are in London? Print the names of all the tube lines.

In [20]:
# One advantage of searching via the mode is that the request accepts an array for an input.
url_lines = "/Line/Mode/tube,bus"
res_lines = re.get(url_root+url_lines+url_append)
total_lines = res_lines.json()
res_lines.status_code

200

In [19]:
JSON(tube_lines)

<IPython.core.display.JSON object>

In [21]:
# I chose to present to use this for loop because our total_lines includes bus and train lines. This was an easy method to return the names of the train lines.
# Instead of printing they could easily have been appended to a list. 

for line in range(len(total_lines)):
    if total_lines[line]['modeName'] == "tube":
        print(total_lines[line]['name'])
print(len(total_lines))
        

Bakerloo
Central
Circle
District
Hammersmith & City
Jubilee
Metropolitan
Northern
Piccadilly
Victoria
Waterloo & City
689


## Task
How many stations are on the `victoria` line?

In [22]:
# Many different search functions will give the list of stations. I chose this one because the data was ordered in station order.

url_victoria = "/Line/Victoria/Route/Sequence/Inbound"
res_victoria = re.get(url_root+url_victoria+url_append)
stations_victoria = res_victoria.json()
res_victoria.status_code

200

In [23]:
JSON(stations_victoria)

<IPython.core.display.JSON object>

In [24]:
print(f"The number of stations on the Victoria line is: {len(stations_victoria['stopPointSequences'][0]['stopPoint'])}")

The number of stations on the Victoria line is: 16


## Task
Plan the journey from Heathrow Airport to Tower Bridge using Bus and Tube? Which way is faster? Example output:
```
Planned duration:
Bus: x minutes
Tube: y minutes
```

We need to search the documentation for correct requests and parameters we need.

In [ ]:
# Due to the large amount of parameters for this request, I found it best to place the parameters I wanted to be able to tune in a seperate parameter list. 

In [78]:
url_root = "https://api.tfl.gov.uk"
url_append = f'?app_id={app_id}&app_key={app_key}'

#Start and Finish require valid stops. You can use exact GPS coordinates, station names or station ID's among others. 

start = ["Heathrow Airport Terminals 1-3, Heathrow Terminals 2 & 3 Rail Station",
         "Hillingdon, Heathrow Airport Terminal 2",
         "Heathrow Airport, Heathrow Airport Terminal 3",
         "Heathrow Airport Terminal 4, Heathrow Terminal 4",
         "Heathrow Airport Terminal 5, Heathrow Terminal 5",
         "Hillingdon, London Heathrow Airport Terminal 5",
         "Hillingdon, Heathrow Airport Terminal 3 Arrivals",
         "Hillingdon, Heathrow Airport Terminal 4 Arrivals",
         "Hillingdon, Heathrow Airport Terminal 5 Arrivals",
         "Sipson, Renaissance Hotel Heathrow",
         "Cranford (London), Travelodge Heathrow Central"]
                 
finish = "City of London, Tower Bridge"

#Terminal 2 is for international arrivals, so that is my startpoint for this journey. 

url_journey = f"/Journey/JourneyResults/{start[1]}/to/{finish}"

# These are parameters I found useful for planning my trip. Not all need to be called at once, but verify url_parameters is correct before continuing.

modes = ["bus","tube","national-rail"]
timeis = "departing"
time = "1200"
date = "20210401"
via = ""
journeyPreference = ["LeastInterchange", "LeastTime", "LeastWalking"]
maxTransferMinutes = '480'
maxWalkingMinutes = '30'
walkingSpeed = ["slow", "average","fast"]
useMultiModalCall = [True, False]
walkingOptimization = [True,False]
routeBetweenEntrances = [True,False]

url_parameters = f"?journeyPreference={journeyPreference[2]}&timeis={timeis}&time={time}&date={date}&maxWalkingMinutes={maxWalkingMinutes}&"


In [79]:
res_journey = re.get(url_root+url_journey+url_parameters+url_append)
res_journey.status_code

200

In [37]:
print(start[8])

Hillingdon, Heathrow Airport Terminal 5 Arrivals


In [74]:
# JSON data for bus departure from terminal 2. 
journey_bus_t2 
#= res_journey.json()
JSON(journey_bus_t2)

<IPython.core.display.JSON object>

In [75]:
print(f"The bus journey is : {journey_bus_t2['journeys'][0]['duration']} minutes.")

The bus journey is : 203 minutes.


In [55]:
#JSON data for tube departure from terminal 2.
journey_bus_tube_t2
#= res_journey.json()
JSON(journey_bus_tube_t2)

<IPython.core.display.JSON object>

In [70]:
print(f"The journey using bus or tube is : {journey_bus_tube_t2['journeys'][0]['duration']} minutes.")

The journey using bus or tube is : 87 minutes.


In [59]:
#JSON data for tube departure from terminal 2.
journey_tube_t2 
#= res_journey.json()
JSON(journey_tube_t2)

<IPython.core.display.JSON object>

In [71]:
print(f"The journey using tube is : {journey_tube_t2['journeys'][0]['duration']} minutes.")

The journey using tube is : 84 minutes.


In [62]:
#JSON data for fastest route from terminal 2.
journey_fastest_t2
#= res_journey.json()
JSON(journey_fastest_t2)

<IPython.core.display.JSON object>

In [66]:
print(f"The fastest journey is : {journey_fastest_t2['journeys'][0]['duration']} minutes.")

The fastest journey is : 76 minutes.


In [80]:
#Json data for least walking route from terminal 2.
journey_leastWalking_t2
#= res_journey.json()
JSON(journey_leastWalking_t2)

<IPython.core.display.JSON object>

In [82]:
print(f"The fastest journey with the least walking is : {journey_leastWalking_t2['journeys'][0]['duration']} minutes.")

The fastest journey with the least walking is : 93 minutes.
